# Set-up

**Setting up the conda environment:**

```
conda create -n sql python
conda activate sql
conda install ipython-sql sqlalchemy psycopg2 notebook pandas -c conda-forge
```

**Dataset:**
- [Cyclistic trip dataset](https://divvy-tripdata.s3.amazonaws.com/index.html)

**License:**
- The data has been made available by Motivate International Inc. under this [license](https://ride.divvybikes.com/data-license-agreement)

## Bike_share

The project started with data cleaning using PostgreSQL

Preload monthly dataset from 2020-04 to 2021-12 in postgresql @ pgAdmin

The following code were used to assess postgresql, analyze, and clean the data.

In [ ]:
%load_ext sql

In [ ]:
import os

In [ ]:
host = "localhost"
database = "google_case_1"
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')

In [ ]:
connection_string = f"postgresql://{user}:{password}@{host}/{database}"

In [ ]:
%sql $connection_string

The following data type are present in the dataset:

- ride_id      =      VARCHAR
- rideable_type  =    VARCHAR
- started_at     =    timestamp
- ended_at       =    timestamp
- start_station_name = VARCHAR
- start_station_id =  VARCHAR
- end_station_name =  VARCHAR
- end_station_id = VARCHAR
- start_lat       =   numeric
- start_lng       =   numeric
- end_lat         =   numeric
- end_lng         =   numeric
- member_casual    =  VARCHAR

Combine all the dataset into a single table without any duplicates using 'UNION'

In [ ]:
%%sql 

CREATE TABLE public.combined_dataset AS(
SELECT * FROM year_202004 UNION SELECT * FROM year_202005 UNION SELECT * FROM year_202006 UNION 
SELECT * FROM year_202007 UNION SELECT * FROM year_202008 UNION SELECT * FROM year_202009 UNION 
SELECT * FROM year_202010 UNION SELECT * FROM year_202011 UNION SELECT * FROM year_202012 UNION 
SELECT * FROM year_202101 UNION SELECT * FROM year_202102 UNION SELECT * FROM year_202103 UNION 
SELECT * FROM year_202104 UNION SELECT * FROM year_202105 UNION SELECT * FROM year_202106 UNION 
SELECT * FROM year_202107 UNION SELECT * FROM year_202108 UNION SELECT * FROM year_202109 UNION 
SELECT * FROM year_202110 UNION SELECT * FROM year_202111 UNION SELECT * FROM year_202112)


Eliminate all rows with null values

In [ ]:
%%sql

CREATE TABLE public.combined_no_null AS(
SELECT * FROM combined_dataset
WHERE 
rideable_type IS NOT NULL AND started_at IS NOT NULL AND
ended_at IS NOT NULL AND start_station_name IS NOT NULL AND 
start_station_id IS NOT NULL AND end_station_name IS NOT NULL AND
end_station_id IS NOT NULL AND start_lat IS NOT NULL AND start_lng IS NOT NULL AND
end_lat IS NOT NULL AND end_lng IS NOT NULL AND member_casual IS NOT NULL);

Eliminate all rows with the 'ended_at' less than 'started_at'

In [ ]:
%%sql

CREATE TABLE cleaned_dataset AS(
SELECT *

FROM combined_no_null

WHERE ended_at > started_at )



The project was limited to only the last one year of data

In [ ]:
%%sql

CREATE TABLE cleaned_dataset_2021 AS(
SELECT *

FROM cleaned_dataset

WHERE started_at BETWEEN '2020-12-31 23:59:59' AND '2022-01-01 00:00:00')


The final clean data was uploaded to Tableau to continue with analysis and visualization